In [1]:
#import krotov
#import qutip as qt# NBVAL_IGNORE_OUTPUT
#import qutip
#import numpy as np
#import scipy
#import matplotlib
#import matplotlib.pylab as plt
#import krotov
import numpy as np
import sympy as sp
from sympy import Function,Symbol,symbols,zeros,Matrix,sqrt,simplify,solve,diff,dsolve,lambdify
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from sympy.physics.quantum.dagger import Dagger

In [2]:
t=Symbol('t')
gamma=Symbol('gamma',Real=True,positive=True)
Nb=Symbol('N_b',Real=True,positive=True)
x = Function('x')(t)
c= Function('c')(t)
x1 = Function('x1')(t)
x2 = Function('x2')(t)
p1= Function('p1')(t)
p2 = Function('p2')(t)
e=Function('e')(t)
mu,g=symbols('mu g')
x11,x12,p11,p12,e1,de1,c1=symbols('x1 x2 p1 p2 e de c')
de=diff(e,t)
F=zeros(2,2)
F[0,1]=1
F[1,0]=-1
B=zeros(2,2)
B[0,0]=1
B[1,1]=-1
#A=np.block([
 #       [g*B+F,             Matrix([[0,0],[0,0]])],
  #      [Matrix([[0,0],[0,0]]), g*B  +F             ]
   #     ])
Omega=sp.Matrix([[0,0,0,0,0],[0,0,0,1,0],[0,0,0,0,1],[0,-1,0,0,0],[0,0,-1,0,0]])

C=sp.Matrix([[0,0,0,0,0],[0,sp.sqrt(gamma*(Nb+1)/2),0,1j*sp.sqrt(gamma*(Nb+1)/2),0],[0,sp.sqrt(gamma*(Nb)/2),0,-1j*sp.sqrt(gamma*(Nb)/2),0],[0,0,0,0,0],[0,0,0,0,0]])
A=sp.Matrix([[0,0,0,0,0],[0,0,0,1,g],[0,0,0,g,1],[0,-1,-g,0,0],[0,-g,-1,0,0]])+Omega*((Dagger(C)*C).as_real_imag()[1])
D=Omega*((Dagger(C)*C).as_real_imag()[0])*Omega.T
d=[]
d.append(c)
d.append(-sqrt(2)*mu*e*c)
d.append(0)
d.append(0)
d.append(0)
v=[]
v.append(c)
v.append(x1)
v.append(x2)
v.append(p1)
v.append(p2)

In [3]:
def to_symbols(expr):
    return expr.subs({x1:x11,x2:x12,p1:p11,p2:p12,e:e1,diff(e,t):de1,c:c1})

In [4]:
v=Matrix(v)
to_symbols(v)

Matrix([
[ c],
[x1],
[x2],
[p1],
[p2]])

In [5]:
r=simplify(diff(v,t)-Matrix(A)*Matrix(v)-Matrix(Omega)*Matrix(d))
r

Matrix([
[                                                            Derivative(c(t), t)],
[                      -g*p2(t) + 0.5*gamma*x1(t) - p1(t) + Derivative(x1(t), t)],
[                                        -g*p1(t) - p2(t) + Derivative(x2(t), t)],
[g*x2(t) + 0.5*gamma*p1(t) - sqrt(2)*mu*c(t)*e(t) + x1(t) + Derivative(p1(t), t)],
[                                         g*x1(t) + x2(t) + Derivative(p2(t), t)]])

In [6]:
vector=diff(Matrix([c,x1,x2,p1,p2]),t)

In [7]:
import sympy
#eqs=list(to_symbols(Matrix(A)*Matrix(v)))
#t=sympy.linear_eq_to_matrix(eqs, [x11,x12,p11,p12,c1])[0]


In [8]:
#t2=sympy.linear_eq_to_matrix(eqs, [x11,x12,p11,p12,c1])[1]

In [9]:
#t*Matrix([x11,x12,p11,p12,c1])

In [10]:
sol=simplify(r-diff(Matrix([c,x1,x2,p1,p2]),t))
sol

Matrix([
[                                                       0],
[                      -g*p2(t) + 0.5*gamma*x1(t) - p1(t)],
[                                        -g*p1(t) - p2(t)],
[g*x2(t) + 0.5*gamma*p1(t) - sqrt(2)*mu*c(t)*e(t) + x1(t)],
[                                         g*x1(t) + x2(t)]])

In [11]:
import sympy
eqs=list(to_symbols(sol))
t=sympy.linear_eq_to_matrix(eqs, [c1,x11,x12,p11,p12])[0]
t2=sympy.linear_eq_to_matrix(eqs, [c1,x11,x12,p11,p12])[1]


In [12]:
t*Matrix([c1,x11,x12,p11,p12])+vector

Matrix([
[                                              Derivative(c(t), t)],
[                 -g*p2 + 0.5*gamma*x1 - p1 + Derivative(x1(t), t)],
[                                -g*p1 - p2 + Derivative(x2(t), t)],
[-sqrt(2)*c*e*mu + g*x2 + 0.5*gamma*p1 + x1 + Derivative(p1(t), t)],
[                                 g*x1 + x2 + Derivative(p2(t), t)]])

In [13]:
def to_func(expr):
    return expr.subs({x11:x1,x12:x2,p11:p1,p12:p2,e1:e,de1:de,c1:c})

In [14]:
new_eqs=to_func(t*Matrix([c1,x11,x12,p11,p12])+vector)
new_eqs

Matrix([
[                                                            Derivative(c(t), t)],
[                      -g*p2(t) + 0.5*gamma*x1(t) - p1(t) + Derivative(x1(t), t)],
[                                        -g*p1(t) - p2(t) + Derivative(x2(t), t)],
[g*x2(t) + 0.5*gamma*p1(t) - sqrt(2)*mu*c(t)*e(t) + x1(t) + Derivative(p1(t), t)],
[                                         g*x1(t) + x2(t) + Derivative(p2(t), t)]])

In [15]:
#field=np.linspace(0,100,1000)
lista=np.random.rand(5)
time,dt=np.linspace(1,10,5,retstep=True)
def field_discrete(lista,t,dt):
    return lista[int(t/dt)]

def derivative(lista,t,dt):
    return np.array([(lista[j]-lista[j-1])/(dt) if (j!=0)&(j!=len(lista)) else lista[j] for j in range(0,len(lista))])[int(t/dt)]

In [16]:
lista

array([0.17899788, 0.90684042, 0.10912275, 0.20034611, 0.59992524])

In [17]:
derivative(lista,time[1],dt)

0.32348557080732193

In [18]:
(lista[1]-lista[0])/dt

0.32348557080732193

In [19]:
field_discrete(lista,10,dt)

0.5999252365457427

In [20]:
#sol=solve([r[0],r[1],r[2],r[3]],diff(x1,t),diff(x2,t),diff(p1,t),diff(p2,t))
#sol

In [21]:
#d1=simplify(sol[diff(x1,t)])
#d2=simplify(sol[diff(x2,t)])
#d3=simplify(sol[diff(p1,t)])
#d4=simplify(sol[diff(p2,t)])


In [22]:
#x1_diff(x1,x2,p1,p2,g,diff(e,t),e,mu)

In [23]:
#p1_diff(x1,x2,p1,p2,g,diff(e,t),e,mu)

In [24]:
#p2_diff(x1,x2,p1,p2,g,diff(e,t),e,mu)

In [25]:
#x2_diff(x1,x2,p1,p2,g,diff(e,t),e,mu)

In [26]:
#x1_diff=lambdify((x1,x2,p1,p2,g,diff(e,t),e,mu),d1)
#x2_diff=lambdify((x1,x2,p1,p2,g,diff(e,t),e,mu),d2)
#p1_diff=lambdify((x1,x2,p1,p2,g,diff(e,t),e,mu),d3)
#p2_diff=lambdify((x1,x2,p1,p2,g,diff(e,t),e,mu),d4)

In [27]:
#x1_diff(x1,x2,g,diff(e,t),e,mu)
##def x1_diff(x1,x2,g,rec,e,mu):
 #   return  x1**2 #x1*x2*g*rec*e*mu
#def x2_diff(x1,x2,g,rec,e,mu):
 #   return  x2*g*rec**2 *e*mu#(x1)**2 + x2 *g*rec*e*mu

In [28]:
#x2_diff(x1,x2,g,diff(e,t),e,mu)

In [29]:
def vectorfield( t,w, p):
    """
    Defines the differential equations for  system.

    Arguments:
        w :  vector of the state variables:
                  w = [x1,x2]
        t :  time
        p: vector of parameters
            p=[g,field,dt,mu]
    """
    x1, x2,p1,p2 = w
    g,field,dt,mu=p
    # Create f = (x1',y1',x2',y2'):
    f = [x1_diff(x1,x2,p1,p2,g,derivative(field,t,dt),field_discrete(field,t,dt),mu),x2_diff(x1,x2,p1,p2,g,derivative(field,t,dt),field_discrete(field,t,dt),mu),
        p1_diff(x1,x2,p1,p2,g,derivative(field,t,dt),field_discrete(field,t,dt),mu),p2_diff(x1,x2,p1,p2,g,derivative(field,t,dt),field_discrete(field,t,dt),mu)]
    return f

In [30]:
#from scipy.integrate import solve_ivp as ivp

#ini=0
#g=0.2
#fin=np.pi/g
#lt=1000
#ini_x1=0.05
#ini_x2=1
#field=np.random.rand(lt)
#np.loadtxt("EFieldValuesFINAL.txt")
#time=np.linspace(ini,fin,lt)
#field=guess_field(time,ini,fin,lt)
#dt=fin/(lt-1)
#p=[g,field,dt,0.1]
#wsol = ivp(vectorfield, [ini,fin],[ini_x1,ini_x1,10,10], args=(p,),t_eval=time)

In [31]:
#wsol.message

In [32]:
#%timeit wsol = ivp(vectorfield, [ini,fin],[ini_x1,ini_x2], args=(p,),t_eval=time)

In [33]:
#from edos import guess_field


In [34]:
#import matplotlib.pyplot as plt
#plt.figure(figsize=(18, 6), dpi=80)
#plt.plot(wsol.t,wsol.y[0],label=r'$x_{1}$')
#plt.plot(wsol.t,wsol.y[1],'-.',label=r'$x_{2}$')
#plt.plot(wsol.t,wsol.y[2],'-.',label=r'$p_{1}$')
#plt.plot(wsol.t,wsol.y[3],'-.',label=r'$p_{2}$')

#plt.legend()
#plt.show()

In [35]:
#Now with CM Matrix

In [36]:
t=Symbol('t')
v11 = Function('v11')(t)
v12 = Function('v12')(t)
v13= Function('v13')(t)
v14 = Function('v14')(t)
v22= Function('v22')(t)
v23 = Function('v23')(t)
v24= Function('v24')(t)
v33 = Function('v33')(t)
v34= Function('v34')(t)
v44 = Function('v44')(t)
x1 = Function('x1')(t)
x2 = Function('x2')(t)
p1= Function('p1')(t)
p2 = Function('p2')(t)
g= symbols('g')
e=Function('e')(t)
mu=symbols('mu')
Omega=sp.Matrix([[0,0,1,0],[0,0,0,1],[-1,0,0,0],[0,-1,0,0]])
C=sp.Matrix([[sp.sqrt(gamma*(Nb+1)/2),0,1j*sp.sqrt(gamma*(Nb+1)/2),0],[sp.sqrt(gamma*(Nb)/2),0,-1j*sp.sqrt(gamma*(Nb)/2),0],[0,0,0,0],[0,0,0,0]])
A=sp.Matrix([[0,0,1,g],[0,0,g,1],[-1,-g,0,0],[-g,-1,0,0]])+Omega*((Dagger(C)*C).as_real_imag()[1])
D=Omega*((Dagger(C)*C).as_real_imag()[0])*Omega.T
#V=Matrix([[v11-2*x1+alpha**2,v12-alpha*x2+beta*x1-alpha*beta,v13-alpha*p1,v14-alpha*p2],[v12-alpha*x2+beta*x1-alpha*beta,v22+beta**2+2*x2,v23+beta*p1,v24+beta*p2],[v13-alpha*p1,v23+beta*p1,v33,v34],[v14-alpha*p2,v24+beta*p2,v34,v44]])
V=Matrix([[v11,v12,v13,v14],[v12,v22,v23,v24],[v13,v23,v33,v34],[v14,v24,v34,v44]])


In [37]:
V

Matrix([
[v11(t), v12(t), v13(t), v14(t)],
[v12(t), v22(t), v23(t), v24(t)],
[v13(t), v23(t), v33(t), v34(t)],
[v14(t), v24(t), v34(t), v44(t)]])

In [38]:
r=simplify(diff(V,t)-Matrix(A)*Matrix(V)-Matrix(V)*Matrix(A).T-D)
simplify(r)

Matrix([
[-1.0*N_b*gamma - 2.0*g*v14(t) + 1.0*gamma*v11(t) - 0.5*gamma - 2.0*v13(t) + 1.0*Derivative(v11(t), t), -g*v13(t) - g*v24(t) + 0.5*gamma*v12(t) - v14(t) - v23(t) + Derivative(v12(t), t),        g*v12(t) - g*v34(t) + gamma*v13(t) + v11(t) - v33(t) + Derivative(v13(t), t), g*v11(t) - g*v44(t) + 0.5*gamma*v14(t) + v12(t) - v34(t) + Derivative(v14(t), t)],
[                    -g*v13(t) - g*v24(t) + 0.5*gamma*v12(t) - v14(t) - v23(t) + Derivative(v12(t), t),                                    -2*g*v23(t) - 2*v24(t) + Derivative(v22(t), t),    g*v22(t) - g*v33(t) + 0.5*gamma*v23(t) + v12(t) - v34(t) + Derivative(v23(t), t),                    g*v12(t) - g*v34(t) + v22(t) - v44(t) + Derivative(v24(t), t)],
[                         g*v12(t) - g*v34(t) + gamma*v13(t) + v11(t) - v33(t) + Derivative(v13(t), t),  g*v22(t) - g*v33(t) + 0.5*gamma*v23(t) + v12(t) - v34(t) + Derivative(v23(t), t), -N_b*gamma + 2*g*v23(t) + gamma*v33(t) - gamma/2 + 2*v13(t) + Derivative(v33(t), t), g*v13(t) 

In [39]:
new_eqs

Matrix([
[                                                            Derivative(c(t), t)],
[                      -g*p2(t) + 0.5*gamma*x1(t) - p1(t) + Derivative(x1(t), t)],
[                                        -g*p1(t) - p2(t) + Derivative(x2(t), t)],
[g*x2(t) + 0.5*gamma*p1(t) - sqrt(2)*mu*c(t)*e(t) + x1(t) + Derivative(p1(t), t)],
[                                         g*x1(t) + x2(t) + Derivative(p2(t), t)]])

In [40]:
c_eqs=[r[0,0],r[0,1],r[0,2],r[0,3],r[1,1],r[1,2],r[1,3],r[2,2],r[2,3],r[3,3]]

In [41]:
gaga=new_eqs-vector
gaga

Matrix([
[                                                       0],
[                      -g*p2(t) + 0.5*gamma*x1(t) - p1(t)],
[                                        -g*p1(t) - p2(t)],
[g*x2(t) + 0.5*gamma*p1(t) - sqrt(2)*mu*c(t)*e(t) + x1(t)],
[                                         g*x1(t) + x2(t)]])

In [42]:
dx1=diff(x1,t)
dx2=diff(x2,t)
dp1=diff(p1,t)
dp2=diff(p2,t)
dc=diff(c,t)


In [43]:
despejadas=[(c_eqs[i]).subs({dc:gaga[0],dx1:gaga[1],dx2:gaga[2],dp1:gaga[3],dp2:gaga[4]}) for i in range(10)]

In [44]:
despejadas=[despejadas[i].simplify() for i in range(10)]

In [45]:
#eqs=list(to_symbols(sol.subs(diff(c,t),0)))
#t=sympy.linear_eq_to_matrix(eqs, [c1,x11,x12,p11,p12])[0]
#t2=sympy.linear_eq_to_matrix(eqs, [c1,x11,x12,p11,p12])[1]


In [46]:
covariance=[v11,v12,v13,v14,v22,v23,v24,v33,v34,v44]
coeffs=[(Matrix(despejadas).expand()[i]).collect(diff(covariance[i],t)).coeff(diff(covariance[i],t)) for i in range(len(despejadas))]
coeffs

[1.00000000000000, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [47]:
despeje=[diff(covariance[i],t)*coeffs[i] for i in range(len(covariance))]

In [48]:
kara=simplify(Matrix(despejadas).expand()-Matrix(despeje))
kara

Matrix([
[-1.0*N_b*gamma - 2.0*g*v14(t) + 1.0*gamma*v11(t) - 0.5*gamma - 2.0*v13(t)],
[                -g*v13(t) - g*v24(t) + 0.5*gamma*v12(t) - v14(t) - v23(t)],
[                     g*v12(t) - g*v34(t) + gamma*v13(t) + v11(t) - v33(t)],
[                 g*v11(t) - g*v44(t) + 0.5*gamma*v14(t) + v12(t) - v34(t)],
[                                                   -2*g*v23(t) - 2*v24(t)],
[                 g*v22(t) - g*v33(t) + 0.5*gamma*v23(t) + v12(t) - v34(t)],
[                                    g*v12(t) - g*v34(t) + v22(t) - v44(t)],
[              -N_b*gamma + 2*g*v23(t) + gamma*v33(t) - gamma/2 + 2*v13(t)],
[                 g*v13(t) + g*v24(t) + 0.5*gamma*v34(t) + v14(t) + v23(t)],
[                                                    2*g*v14(t) + 2*v24(t)]])

In [49]:
v111,v112,v113,v114 = symbols('v_11:15')
v122,v123,v124 = symbols('v_22:25')
v133,v134=symbols('v_33 v_34')
v144=symbols('v_44')

In [50]:
subsss={v11:v111,v12:v112,v13:v113,v14:v114,v22:v122,v23:v123,v24:v124,v33:v133,v34:v134,v44:v144,diff(c1,t):0,(c1**2):c1}

In [51]:
to_symbols(kara.subs(subsss))

Matrix([
[-1.0*N_b*gamma - 2.0*g*v_14 + 1.0*gamma*v_11 - 0.5*gamma - 2.0*v_13],
[                    -g*v_13 - g*v_24 + 0.5*gamma*v_12 - v_14 - v_23],
[                         g*v_12 - g*v_34 + gamma*v_13 + v_11 - v_33],
[                     g*v_11 - g*v_44 + 0.5*gamma*v_14 + v_12 - v_34],
[                                                 -2*g*v_23 - 2*v_24],
[                     g*v_22 - g*v_33 + 0.5*gamma*v_23 + v_12 - v_34],
[                                      g*v_12 - g*v_34 + v_22 - v_44],
[              -N_b*gamma + 2*g*v_23 + gamma*v_33 - gamma/2 + 2*v_13],
[                     g*v_13 + g*v_24 + 0.5*gamma*v_34 + v_14 + v_23],
[                                                  2*g*v_14 + 2*v_24]])

In [52]:
full_dvect=Matrix(list(vector)+list(diff(Matrix(covariance),t)))
full_dvect

Matrix([
[  Derivative(c(t), t)],
[ Derivative(x1(t), t)],
[ Derivative(x2(t), t)],
[ Derivative(p1(t), t)],
[ Derivative(p2(t), t)],
[Derivative(v11(t), t)],
[Derivative(v12(t), t)],
[Derivative(v13(t), t)],
[Derivative(v14(t), t)],
[Derivative(v22(t), t)],
[Derivative(v23(t), t)],
[Derivative(v24(t), t)],
[Derivative(v33(t), t)],
[Derivative(v34(t), t)],
[Derivative(v44(t), t)]])

In [53]:
full_vect=Matrix([c1,x11,x12,p11,p12,v111,v112,v113,v114,v122,v123,v124,v133,v134,v144])
full_vect

Matrix([
[   c],
[  x1],
[  x2],
[  p1],
[  p2],
[v_11],
[v_12],
[v_13],
[v_14],
[v_22],
[v_23],
[v_24],
[v_33],
[v_34],
[v_44]])

In [54]:
full_eqs=Matrix(list(to_symbols(sol))+list(to_symbols(kara.subs(subsss))))

In [55]:
tocheck=[full_eqs[i].expand().subs(Nb,Nb*c1).subs(c1**2,c1).subs(c1*x12,x12).subs(c1*x11,x11).subs(c1*p11,p11).subs(c1*p12,p12) for i in range(len(full_eqs))]
Matrix(tocheck)

Matrix([
[                                                                    0],
[                                            -g*p2 + 0.5*gamma*x1 - p1],
[                                                           -g*p1 - p2],
[                           -sqrt(2)*c*e*mu + g*x2 + 0.5*gamma*p1 + x1],
[                                                            g*x1 + x2],
[-1.0*N_b*c*gamma - 2.0*g*v_14 + 1.0*gamma*v_11 - 0.5*gamma - 2.0*v_13],
[                      -g*v_13 - g*v_24 + 0.5*gamma*v_12 - v_14 - v_23],
[                           g*v_12 - g*v_34 + gamma*v_13 + v_11 - v_33],
[                       g*v_11 - g*v_44 + 0.5*gamma*v_14 + v_12 - v_34],
[                                                   -2*g*v_23 - 2*v_24],
[                       g*v_22 - g*v_33 + 0.5*gamma*v_23 + v_12 - v_34],
[                                        g*v_12 - g*v_34 + v_22 - v_44],
[              -N_b*c*gamma + 2*g*v_23 + gamma*v_33 - gamma/2 + 2*v_13],
[                       g*v_13 + g*v_24 + 

In [56]:
len(tocheck)

15

In [57]:
t=sympy.linear_eq_to_matrix(tocheck, list(full_vect))[0]
t2=sympy.linear_eq_to_matrix(tocheck, list(full_vect))[1]

In [58]:
vector


Matrix([
[ Derivative(c(t), t)],
[Derivative(x1(t), t)],
[Derivative(x2(t), t)],
[Derivative(p1(t), t)],
[Derivative(p2(t), t)]])

In [59]:
sol

Matrix([
[                                                       0],
[                      -g*p2(t) + 0.5*gamma*x1(t) - p1(t)],
[                                        -g*p1(t) - p2(t)],
[g*x2(t) + 0.5*gamma*p1(t) - sqrt(2)*mu*c(t)*e(t) + x1(t)],
[                                         g*x1(t) + x2(t)]])

In [60]:
full_vect[11]

v_24

In [61]:
((t*full_vect)[11]).subs({p12:0,x11:0,p11:0,v122:0,x12:0,v114:0,v124:0}).subs({e1:e,de1:de}).factor()

g*v_12 - g*v_34 - v_44

In [62]:
ricard=r[1,3].subs(vector[4],sol[4]).expand().collect(covariance[4]).collect(p1).collect(p2).subs(c*x1,x1).collect(x1).subs(c*x2,x2).collect(x2).collect(v12).subs(vector[0],0).subs(c**2,c).collect(c)
simplify(ricard.subs({p2:0,x1:0,p1:0,full_dvect[11]:0,v22:0,x2:0,v14:0,v24:0})).expand().factor()

g*v12(t) - g*v34(t) - v44(t)

In [63]:
lista=[]
for i in range(15):
    dummy=[simplify(t[i,j].subs({e1:e,de1:de})) for j in range(15)]
    lista.append(dummy)

In [64]:
matriz=Matrix(lista)
matriz

Matrix([
[               0,         0, 0,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[               0, 0.5*gamma, 0,        -1, -g,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[               0,         0, 0,        -g, -1,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[-sqrt(2)*mu*e(t),         1, g, 0.5*gamma,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[               0,         g, 1,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[  -1.0*N_b*gamma,         0, 0,         0,  0, 1.0*gamma,         0,  -2.0,    -2.0*g, 0,         0,  0,     0,         0,  0],
[               0,         0, 0,         0,  0,         0, 0.5*gamma,    -g,        -1, 0,        -1, -g,     0,         0,  0],
[               0,         0, 0,         0,  0,         1,         g, gamma,         0, 

In [65]:
matriz[5,0]+=-gamma/2
matriz[12,0]+=-gamma/2
matriz

Matrix([
[                       0,         0, 0,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                       0, 0.5*gamma, 0,        -1, -g,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                       0,         0, 0,        -g, -1,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[        -sqrt(2)*mu*e(t),         1, g, 0.5*gamma,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                       0,         g, 1,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[-1.0*N_b*gamma - gamma/2,         0, 0,         0,  0, 1.0*gamma,         0,  -2.0,    -2.0*g, 0,         0,  0,     0,         0,  0],
[                       0,         0, 0,         0,  0,         0, 0.5*gamma,    -g,        -1, 0,        -1, -g,     0,         0,  0],
[                       0,      

In [66]:
t0=Symbol("t0")
A=matriz[5:,5:]
A

Matrix([
[1.0*gamma,         0,  -2.0,    -2.0*g, 0,         0,  0,     0,         0,  0],
[        0, 0.5*gamma,    -g,        -1, 0,        -1, -g,     0,         0,  0],
[        1,         g, gamma,         0, 0,         0,  0,    -1,        -g,  0],
[        g,         1,     0, 0.5*gamma, 0,         0,  0,     0,        -1, -g],
[        0,         0,     0,         0, 0,      -2*g, -2,     0,         0,  0],
[        0,         1,     0,         0, g, 0.5*gamma,  0,    -g,        -1,  0],
[        0,         g,     0,         0, 1,         0,  0,     0,        -g, -1],
[        0,         0,     2,         0, 0,       2*g,  0, gamma,         0,  0],
[        0,         0,     g,         1, 0,         1,  g,     0, 0.5*gamma,  0],
[        0,         0,     0,       2*g, 0,         0,  2,     0,         0,  0]])

In [67]:
matriz*full_vect

Matrix([
[                                                                    0],
[                                            -g*p2 + 0.5*gamma*x1 - p1],
[                                                           -g*p1 - p2],
[                        -sqrt(2)*c*mu*e(t) + g*x2 + 0.5*gamma*p1 + x1],
[                                                            g*x1 + x2],
[c*(-1.0*N_b*gamma - gamma/2) - 2.0*g*v_14 + 1.0*gamma*v_11 - 2.0*v_13],
[                      -g*v_13 - g*v_24 + 0.5*gamma*v_12 - v_14 - v_23],
[                           g*v_12 - g*v_34 + gamma*v_13 + v_11 - v_33],
[                       g*v_11 - g*v_44 + 0.5*gamma*v_14 + v_12 - v_34],
[                                                   -2*g*v_23 - 2*v_24],
[                       g*v_22 - g*v_33 + 0.5*gamma*v_23 + v_12 - v_34],
[                                        g*v_12 - g*v_34 + v_22 - v_44],
[            c*(-N_b*gamma - gamma/2) + 2*g*v_23 + gamma*v_33 + 2*v_13],
[                       g*v_13 + g*v_24 + 

In [68]:
Matrix(tocheck)

Matrix([
[                                                                    0],
[                                            -g*p2 + 0.5*gamma*x1 - p1],
[                                                           -g*p1 - p2],
[                           -sqrt(2)*c*e*mu + g*x2 + 0.5*gamma*p1 + x1],
[                                                            g*x1 + x2],
[-1.0*N_b*c*gamma - 2.0*g*v_14 + 1.0*gamma*v_11 - 0.5*gamma - 2.0*v_13],
[                      -g*v_13 - g*v_24 + 0.5*gamma*v_12 - v_14 - v_23],
[                           g*v_12 - g*v_34 + gamma*v_13 + v_11 - v_33],
[                       g*v_11 - g*v_44 + 0.5*gamma*v_14 + v_12 - v_34],
[                                                   -2*g*v_23 - 2*v_24],
[                       g*v_22 - g*v_33 + 0.5*gamma*v_23 + v_12 - v_34],
[                                        g*v_12 - g*v_34 + v_22 - v_44],
[              -N_b*c*gamma + 2*g*v_23 + gamma*v_33 - gamma/2 + 2*v_13],
[                       g*v_13 + g*v_24 + 

In [69]:
H0=simplify(matriz.subs({e:0}))
He=simplify(matriz.subs({de:0})-H0)

In [70]:
He=He/e

In [71]:
H0

Matrix([
[                     0,         0, 0,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                     0, 0.5*gamma, 0,        -1, -g,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                     0,         0, 0,        -g, -1,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                     0,         1, g, 0.5*gamma,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                     0,         g, 1,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[-gamma*(1.0*N_b + 0.5),         0, 0,         0,  0, 1.0*gamma,         0,  -2.0,    -2.0*g, 0,         0,  0,     0,         0,  0],
[                     0,         0, 0,         0,  0,         0, 0.5*gamma,    -g,        -1, 0,        -1, -g,     0,         0,  0],
[                     0,         0, 0,        

In [83]:
He

Matrix([
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-sqrt(2)*mu, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[          0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [84]:
hlineal=lambdify((mu),He)
import dill
dill.settings['recurse'] = True

In [85]:
dill.dump(hlineal, open("H_campo", "wb"))


In [75]:
Hde=Hde/de

NameError: name 'Hde' is not defined

In [ ]:
Hde=simplify(Hde)

In [ ]:
Hde

In [ ]:
func=lambdify((mu,g),Hde)

In [ ]:
Matrix(func(2,0.1))

In [ ]:
H0

In [97]:
h0lineal=lambdify((g,gamma,Nb),H0)
h0lineal(0.2,1,0.5)

array([[ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.5,  0. , -1. , -0.2,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. , -0.2, -1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0. ,  1. ,  0.2,  0.5,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.2,  1. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [-1. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0. , -2. , -0.4,  0. ,  0. ,
         0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0.5, -0.2, -1. ,  0. , -1. ,
        -0.2,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  1. ,  0.2,  1. ,  0. ,  0. ,  0. ,
         0. , -1. , -0.2,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  1. ,  0. ,  0.5,  0. ,  0. ,
         0. ,  0. , -1. , -0.2],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,

In [98]:
dill.dump(h0lineal, open("H_0", "wb"))


In [ ]:
H0

In [ ]:
from Integrals import overlap2

In [ ]:
vector1=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
vector2=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]


In [ ]:
V=[[vector1[5],vector1[6],vector1[7],vector1[8]],[1,vector1[9],vector1[10],vector1[11]],[1,1,vector1[12],vector1[13]],[1,1,1,vector1[14]]]
R=[vector1[i] for i in range(1,5)]
B=[[vector2[5],vector2[6],vector2[7],vector2[8]],[1,vector2[9],vector2[10],vector2[11]],[1,1,vector2[12],vector2[13]],[1,1,1,vector2[14]]]
S=[vector2[i] for i in range(1,5)]

In [ ]:
V

In [ ]:
R

In [ ]:
B

In [ ]:
S

In [ ]:
overlap2(vector1,vector2)

In [ ]:
(e*de)

In [ ]:
sp.sin(2)*sp.cos(2)

In [72]:
matriz

Matrix([
[                       0,         0, 0,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                       0, 0.5*gamma, 0,        -1, -g,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                       0,         0, 0,        -g, -1,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[        -sqrt(2)*mu*e(t),         1, g, 0.5*gamma,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[                       0,         g, 1,         0,  0,         0,         0,     0,         0, 0,         0,  0,     0,         0,  0],
[-1.0*N_b*gamma - gamma/2,         0, 0,         0,  0, 1.0*gamma,         0,  -2.0,    -2.0*g, 0,         0,  0,     0,         0,  0],
[                       0,         0, 0,         0,  0,         0, 0.5*gamma,    -g,        -1, 0,        -1, -g,     0,         0,  0],
[                       0,      

In [73]:
from sympy import mathematica_code as mcode, symbols, sin

In [74]:
mcode(matriz)

'{{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, {0, 0.5*gamma, 0, -1, -g, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, {0, 0, 0, -g, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, {-2^(1/2)*mu*e[t], 1, g, 0.5*gamma, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, {0, g, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0}, {-1.0*N_b*gamma - 1/2*gamma, 0, 0, 0, 0, 1.0*gamma, 0, -2.0, -2.0*g, 0, 0, 0, 0, 0, 0}, {0, 0, 0, 0, 0, 0, 0.5*gamma, -g, -1, 0, -1, -g, 0, 0, 0}, {0, 0, 0, 0, 0, 1, g, gamma, 0, 0, 0, 0, -1, -g, 0}, {0, 0, 0, 0, 0, g, 1, 0, 0.5*gamma, 0, 0, 0, 0, -1, -g}, {0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2*g, -2, 0, 0, 0}, {0, 0, 0, 0, 0, 0, 1, 0, 0, g, 0.5*gamma, 0, -g, -1, 0}, {0, 0, 0, 0, 0, 0, g, 0, 0, 1, 0, 0, 0, -g, -1}, {-N_b*gamma - 1/2*gamma, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2*g, 0, gamma, 0, 0}, {0, 0, 0, 0, 0, 0, 0, g, 1, 0, 1, g, 0, 0.5*gamma, 0}, {0, 0, 0, 0, 0, 0, 0, 0, 2*g, 0, 0, 2, 0, 0, 0}}'